# Задача 1

Есть файл с именами (names.txt), нужно выполнить следующие действия и посчитать результат:  
1. Отсортировать все имена в лексикографическом порядке;  
2. Посчитать для каждого имени алфавитную сумму – сумму порядковых номеров букв (MAY: 13 + 1 + 25 = 39);  
3. Умножить алфавитную сумму каждого имени на порядковый номер имени в отсортированном списке (индексация начинается с 1). Например, если MAY находится на 63 месте в списке, то результат для него будет 63 * 39 = 2457;  
4. Просуммировать произведения из пункта 3 для всех имен из файла и получить число. Это число и будет ответом.

In [1]:
import pandas as pd

In [2]:
with open('names.txt') as f:
    text = f.read()
names = [name[1:-1] for name in text.split(',')]

In [3]:
data = pd.DataFrame(names, columns=['name'])
data = data.sort_values(by='name').reset_index(drop=True).reset_index()
data['index'] = data['index'] + 1
data.columns = ['pos', 'name']

In [4]:
num_to_let = {key: value for (key, value) in enumerate('ABCDEFGHIJKLMNOPQRSTUWVXYZ', 1)}
let_to_nam = {v: k for k, v in num_to_let.items()}

In [5]:
def counter(name):
    s = 0
    for i in name:
        s += let_to_nam[i]
    return(s)

In [6]:
data['sum'] = data['name'].apply(counter)
data['mult'] = data['pos'] * data['sum']

In [7]:
data['mult'].sum()

872299863

# Задача 2

Есть файл со строками (hist.txt) вида: ```<host>\t<ip>\t<page>\n```. Нужно вывести 5 айпи-адресов, которые встречаются чаще других.

In [8]:
import re
from collections import Counter

In [9]:
with open('hits.txt') as f:
    lines = [line.strip() for line in f]

In [10]:
pattern = '\t(.*?)\t'

In [11]:
ip = [re.search(pattern, line).group()[1:-1] for line in lines]

In [12]:
c = Counter()

In [13]:
for num in ip:
    c[num] += 1

In [14]:
c.most_common(5)

[('154.157.157.156', 1531),
 ('82.146.232.163', 1505),
 ('194.78.107.33', 1494),
 ('226.247.119.128', 1494),
 ('21.143.243.182', 1479)]

# Задача 3

В школе ученики и репетиторы занимаются в специальном онлайн-классе, в котором они могут общаться друг с другом, рисовать на доске, переписываться в чате, обмениваться файлами и решать различные задачки. Информациz о каждом уроке записывается в базу данных в таблицу lessons. На каждом таком уроке присутствует один репетитор и один ученик (существуют еще тренировочные уроки, у которых единственный участник – репетитор, но для нашей задачи они неинтересны). В конце урока, когда учитель и ученик покидают класс, они могут оценить (а могут и пропустить этот этап) качество пройденного урока по пятибальной шкале. Все оценки записываются в таблицу quality, чтобы мы могли узнавать, хорошо ли работает наш сайт.

**Нужно в каждый день найти репетитора, уроки которого получили самую низкую оценку качества, и вывести эту оценку.**

lessons.txt содержит следующие поля:  
id – уникальный идентификатор урока  
event_id – идентификатор, связывающий уроки с файлом participants; у нескольких уроков может быть один и тот же event_id  
subject – предмет урока, просто строка  
scheduled_time – время начала урока, в формате ГГГГ-ММ-ДД чч:мм:сс (иногда есть еще миллисекунды); время указано в UTC  

quality.txt содержит следующие поля:  
lesson_id – идентификатор урока, указывает на запись в таблице lessons.txt, у нескольких строчек из этого файла может быть один и тот же lesson_id, потому что оценок за урок может быть несколько (например, оба участника, ученик и репетитор могут оценить качество урока)  
tech_quality – оценка качества урока, это число от 1 до 5; иногда его может не быть, если пользователь не выставил оценку  

users.txt хранит информацию о пользователях и содержит два поля:
id – уникальный идентификатор пользователя  
role – указывает является ли пользователь учеником (pupil) или учителем (tutor)  

participants.txt позволяет связать урок с его участниками, он содержит следующие поля:  
user_id – идентификатор пользователя (указывает на запись в файле users.txt)  
event_id – идентификатор, связывающий урок с участником. То есть, чтобы понять, какие пользователи были на уроке Х, нужно найти в файле participants.txt строки, у которых event_id совпадает с event_id урока Х  

Для решения задачки нужно сделать следующее:
1. Найти все уроки по физике (subject=phys).
2. В каждый день (начало и конец дня считается по московскому времени, то есть UTC+3:00) для каждого репетитора посчитать среднюю арифметическую оценку за его уроки (учитывать только уроки из пункта 1). То есть, если учитель проводил в этот день три урока по физике, один из них он оценил на 3, а ученик оценил его на 4, второму уроку оценку поставил только ученик, и эта оценка 5, а третий урок вообще никто не оценил, то средняя арифметическая оценка учителя за уроки = (3 + 4 + 5) / 3 = 4.
3. Найти учителя, который в этот день имеет самую низкую среднюю арифметическую оценку за уроки (среди всех учителей, проводивших уроки по физике в этот день).
4. Вывести его в формате “<день> <id учителя> <средняя арифметическая оценка>“. Оценку можно округлить с точностью до двух знаков после запятой. Например:  
```2020-01-11 73c9af08-8581-430c-a590-9888ab36deb3 3.67```  
```2020-01-12 909c2c8e-c054-4e9f-a51a-50bf5660f364 3.25```  
```...```
4. Учитывать нужно только тех учителей, за уроки по физике которых в этот день стоит хотя бы одна оценка. Если у нескольких учителей одна и та же самая оценка, можно вывести любого.
5. Если есть необходимость, предварительно можно предобработать данные (сконвертировав их в csv или загрузив в базу данных) и работать уже с ними.

In [15]:
from datetime import datetime
from datetime import timedelta

In [16]:
lessons = pd.read_csv('tech_quality/lessons.csv')
participants = pd.read_csv('tech_quality/participants.csv')
quality = pd.read_csv('tech_quality/quality.csv')
users = pd.read_csv('tech_quality/users.csv')

In [17]:
quality = quality.iloc[:365]
quality = quality.groupby(by='lesson_id').mean() # Средняя оценка за каждый урок
quality.reset_index(inplace=True)
quality.columns = ['id', 'tech_quality']

In [18]:
tutors = users[users['role'] == 'tutor'] # Выберем только учителей
tutors.columns = ['user_id', 'role']
tutors = tutors.groupby(by='user_id').first().reset_index() # Уберём дубликаты

In [19]:
events = pd.merge(participants, tutors, how='left', on='user_id') # Поставим к номерам уроков id учителей
events = events[events['role'] == 'tutor'] # Оставим только учителей 
events['event_id'] = events['event_id'].astype('float64')

In [20]:
phys = lessons[lessons['subject'] == 'phys'].reset_index(drop=True)
phys['scheduled_time'] = phys['scheduled_time'].apply(lambda x: x[:18]) # Уберём миллисекунды, если они есть
phys['scheduled_time'] = phys['scheduled_time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d%H:%M:%S')) # Удобный формат
phys['scheduled_time'] = phys['scheduled_time'] + timedelta(hours=3) # Прибавим 3 часа (московское время)
phys['scheduled_time'] = phys['scheduled_time'].apply(lambda x: x.date()) # Оставим только даты

In [21]:
phys = pd.merge(phys, quality, how='left', on='id') # Поставили оценки за каждый урок по физике

In [22]:
final = pd.merge(phys, events, how='left', on='event_id')
final = final.groupby(by='id').first()
final.reset_index(inplace=True)
final.drop(['subject', 'role'], axis=1, inplace=True)

In [23]:
dates = final['scheduled_time'].unique().tolist()
dates.sort()

In [24]:
for i in dates:
    day_lessons = final[final['scheduled_time'] == i]
    day_lessons = day_lessons.groupby(by='user_id')['tech_quality'].mean()
    day_lessons = day_lessons.reset_index()
    day_lessons = day_lessons[day_lessons['tech_quality'].notna()]
    m = day_lessons['tech_quality'].min()
    print(i, day_lessons[day_lessons['tech_quality'] == m].iloc[0].user_id, m)

2020-01-11 8fe03f08-8581-430c-a590-9888ab36deb3 4.5
2020-01-12 696c838e-c054-4e9f-a51a-50bf5660f364 4.9
2020-01-13 2fa2ab62-f1b0-4036-872f-bcfd9a8686ff 5.0
2020-01-14 c6718d0e-976c-4d6c-b0e0-32c770776567 4.0
2020-01-15 603b8641-c6f6-4d89-ac89-88e50d45aa0d 5.0
2020-01-16 2fa2ab62-f1b0-4036-872f-bcfd9a8686ff 4.0
2020-01-17 696c838e-c054-4e9f-a51a-50bf5660f364 4.5
2020-01-18 43efce48-94b2-4412-857f-223d45969008 4.0
2020-01-19 be676776-8366-4c71-8a35-d58014806eb5 4.5
2020-01-20 30a19496-bdaf-461c-abbc-2709ae520201 4.5
